In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns 

# Bootstrapping/resampling 403 AND THEN scaling up to 7.5 million

In [ ]:
###bootstrap to 403 

#import dataframes that will allow us to scale the sample by square footage
scale = pd.read_csv("")
metaData = pd.read_csv("",
                       usecols=['Identifier','Floor_Area__ft2_']).reset_index()


identifiers = metaData.Identifier.unique()

#####variables######
num_homes = 7500000
nBoot = 100 #number of times we want to bootstap 

###########
finalFinalDF = pd.DataFrame()

for k in range(nBoot):
    print(str(k+1))
    bootDF = pd.DataFrame()
    
    newIndex = np.random.choice(metaData['index'],size=len(metaData['index']),replace=True)
    unique, counts = np.unique(metaData.index[newIndex], return_counts = True)
                                     
    countsDF = pd.DataFrame({'indexMerge':unique,'count_scale':counts})
    
    finalDF = metaData.merge(countsDF,how='left',left_on='index',right_on='indexMerge')                            
    finalDF['boostrap_run'] = k+1
    finalDF['count_scale'] = finalDF.count_scale.fillna(0)                            

    finalDF.to_csv()
    print('saved weighting')
    finalFinalDF = pd.concat([finalFinalDF,finalDF])

    
    finalDF = finalDF.set_index('Identifier')
    for year in [2016,2017,2018,2019]: 
        print(str(year))
        for hour in [1,2,3,4]:
            lolp = pd.read_csv()
            
            overrideFinal = pd.DataFrame()
            noOverrideFinal = pd.DataFrame()
    
            override= pd.DataFrame()
            noOverride = pd.DataFrame()
    
            count = 0
            for low, high, percent in zip(scale.more_than, scale.less_than, scale.percentage):
                overrideFilter = pd.DataFrame()
                noOverrideFilter = pd.DataFrame()
                scaleUp = percent * num_homes
        
                #get the homes from the metadata with the size we want 
                sample = metaData[(metaData.Floor_Area__ft2_ >= low) & (metaData.Floor_Area__ft2_ < high)].reset_index() #identifiers in group
                sampleHomes = finalDF[(finalDF.count_scale >0)&(finalDF.index.isin(sample.Identifier))] #get weighting from the identifiers 
                sampleNum = sampleHomes.count_scale.sum() # number of homes in group (i.e. number of times the homes are weighted)


                for identifier in sample.Identifier:
                    homeDF = pd.read_csv()

                    scaleID = finalDF.loc[identifier,'count_scale'] 
                    
                    overrideFilter  = overrideFilter.add(pd.DataFrame(homeDF['observed_impact_kW_mean']*-1*scaleID),fill_value=0)
                    noOverrideFilter  = noOverrideFilter.add(pd.DataFrame(homeDF['counterfactual_impact_kW_mean']*-1*scaleID),fill_value=0)
        
    
                noOverride = noOverride.add(pd.DataFrame(noOverrideFilter['counterfactual_impact_kW_mean']/1000 * percent * num_homes/sampleNum),fill_value=0)
                override = override.add(pd.DataFrame(overrideFilter['observed_impact_kW_mean']/1000 * percent * num_homes/sampleNum),fill_value=0) 
        

            overrideFinal = lolp.merge(override,how='left',left_index=True,right_index=True)
            overrideFinal = overrideFinal.rename(columns={'observed_impact_kW_mean':'delta_override'})
            overrideFinal.to_csv()
    
            noOverrideFinal = lolp.merge(noOverride,how='left',left_index=True,right_index=True)
            noOverrideFinal = noOverrideFinal.rename(columns={'counterfactual_impact_kW_mean':'delta_noOverride'})
            noOverrideFinal.to_csv()
    
finalFinalDF.to_csv()